# Decision Trees

Decision trees are a classifier in machine learning that allows us to make predictions based on previous data. They are like a series of sequential “if … then” statements you feed new data into to get a result.

Decision trees are easily created, visualized, and interpreted. Because of this, they are typically the first method used to model a dataset. The hierarchical structure and categorical nature of a decision tree makes it highly intuitive to implement.

## Example
To demonstrate decision trees, let’s take a look at an example. Imagine we want to predict whether Mike is going to go grocery shopping on any given day. We can look at previous factors that led Mike to go to the store:

<img src="image/ex.png" width="350">


## Our first split
Here we can see the amount of grocery supplies Mike had, the weather, and whether Mike worked each day. Green rows are days he went to the store, and red days are those he didn’t. The goal of a decision tree is to try to understand why Mike goes to the store, and apply that to new data later on.

Let’s divide the first attribute up into a tree. Mike can either have a low, medium, or high amount of supplies:

<img src="image/decision_tree_1.png" width="550">


Here we can see that Mike never goes to the store if he has a high amount of supplies. This is called a pure subset, a subset with only positive or only negative examples.With decision trees, there is no need to break a pure subset down further.

## Our second split

Let’s break the Med Supplies category into whether Mike worked that day:

<img src="image/decision_tree_2.png" width="550">


Here we can see we have two more pure subsets, so this tree is complete. We can replace any pure subsets with their respective answer - in this case, yes or no.

## Our third split

Finally, let’s split the Low Supplies category by the Weather attribute:
    
<img src="image/decision_tree_3.png" width="650">


**Let's predict for this example : ['low',  'raining', 'yes']**

# Classification and Regression Trees

Decision tree algorithms are also known as CART, or Classification and Regression Trees. A Classification Tree, like the one shown above, is used to get a result from a set of possible values. A Regression Tree is a decision tree where the result is a continuous value, such as the price of a car.



# Splitting (Induction)

Decision trees are created through a process of splitting called induction, but how do we know when to split? We need a recursive algorithm that determines the best attributes to split on. One such algorithm is the greedy algorithm:

1. Starting from the root, we create a split for each attribute.
2. For each created split, calculate the cost of the split.
3. Choose the split that costs the least.
4. Recurse into the sub-trees and continue from step 1.

This process is repeated until all nodes have the same value as the target result, or splitting adds no value to a prediction. This algorithm has the root node as the best classifier.

# Cost of Splitting
The cost of a split is determined by a cost function. The goal of using a cost function is to split the data in a way that can be computed and that provides the most information gain.

For classification trees, those that provide an answer rather than a value, we can compute imformation gain using Gini Impurities:

#### Gini Impurity Function:

<img src='image/Gini_Impurity.png' width='120'>

#### Gini Information Gain Formula:

<img src='image/Gini_Information_Gain.png' width='250'>

To calculate information gain, we first start by computing the Gini Impurity of our root node. Let's take a look at the data we used earlier (this is example not the earlier data):

<img src="image/example.png" width="550">


**Our root node is the target variable (shopped).** To calculate its Gini Impurity, we need to find the sum of probabilities squared for each outcome and subtract this result from one:

<img src="image/Gini_2.png" width="450">

Let's calculate the Gini Information Gain if we split on the first attribute, Supplies. We have three different categories we can split by - Low, Med, and High. For each of these, we calculate its Gini Impurity:

<img src="image/Gini_4.png" width="450">

As you can see, the impurity for High supplies is 0. This means that if we split on Supplies and receive High input, we immediately know what the outcome will be. To determine the Gini Information Gain for this split, we compute the root's impurity minus the weighted average of each child's impurity:

<img src="image/Gini_7.png" width="550">

We continue this pattern for every possible split, then choose the split that gives us the highest information gain value. Maximizing information gain leaves us with the most polarized splits possible, lowering the probability new input is incorrectly classified.

# Implementation

In [1]:
import graphviz
import itertools
import random 
from sklearn.tree import DecisionTreeClassifier, export_graphviz, DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder

In [2]:
# The possible values for each class 
classes = {
    'supplies': ['low', 'med', 'high'],
    'weather':  ['raining', 'cloudy', 'sunny'],
    'worked?':  ['yes', 'no']
}

# Our example data from the documentation
data = [
    ['low',  'sunny',   'yes'],
    ['high', 'sunny',   'yes'],
    ['med',  'cloudy',  'yes'],
    ['low',  'raining', 'yes'],
    ['low',  'cloudy',  'no' ],
    ['high', 'sunny',   'no' ],
    ['high', 'raining', 'no' ],
    ['med',  'cloudy',  'yes'],
    ['low',  'raining', 'yes'],
    ['low',  'raining', 'no' ],
    ['med',  'sunny',   'no' ],
    ['high', 'sunny',   'yes']
]

# Our target variable, whether someone went shopping
target = ['yes', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no']

In [3]:
data, target

([['low', 'sunny', 'yes'],
  ['high', 'sunny', 'yes'],
  ['med', 'cloudy', 'yes'],
  ['low', 'raining', 'yes'],
  ['low', 'cloudy', 'no'],
  ['high', 'sunny', 'no'],
  ['high', 'raining', 'no'],
  ['med', 'cloudy', 'yes'],
  ['low', 'raining', 'yes'],
  ['low', 'raining', 'no'],
  ['med', 'sunny', 'no'],
  ['high', 'sunny', 'yes']],
 ['yes', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no'])

In [4]:
categories = [classes['supplies'], classes['weather'], classes['worked?']]
encoder = OneHotEncoder(categories=categories)
# encoder
x_data = encoder.fit_transform(data)

In [5]:
x_data.toarray()

array([[1., 0., 0., 0., 0., 1., 1., 0.],
       [0., 0., 1., 0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 1., 0., 1., 0.],
       [1., 0., 0., 1., 0., 0., 1., 0.],
       [1., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 0., 1., 0., 1.],
       [0., 0., 1., 1., 0., 0., 0., 1.],
       [0., 1., 0., 0., 1., 0., 1., 0.],
       [1., 0., 0., 1., 0., 0., 1., 0.],
       [1., 0., 0., 1., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 1., 0., 1.],
       [0., 0., 1., 0., 0., 1., 1., 0.]])

In [6]:
# Form and fit our decision tree to the now-encoded data
classifier = DecisionTreeClassifier()
tree = classifier.fit(x_data, target)

In [7]:
# Use our tree to predict the outcome of the random values
prediction_results = tree.predict(encoder.transform(data))

print(prediction_results)

['yes' 'no' 'no' 'no' 'yes' 'no' 'no' 'no' 'no' 'yes' 'yes' 'no']


In [8]:
feature_names = (
    ['supplies=' + x for x in classes["supplies"]] +
    ['weather=' + x for x in classes["weather"]] +
    ['worked=' + x for x in classes["worked?"]]
)
feature_names

['supplies=low',
 'supplies=med',
 'supplies=high',
 'weather=raining',
 'weather=cloudy',
 'weather=sunny',
 'worked=yes',
 'worked=no']

In [11]:
# Shows a visualization of the decision tree using graphviz
from sklearn.tree import export_graphviz 
# Note that sklearn is unable to generate non-binary trees, so these are based on individual options in each class
dot_data = export_graphviz(tree, filled=True, proportion=True, feature_names=feature_names) 
print(dot_data)
graph = graphviz.Source(dot_data)
graph.render(filename='decision_tree', cleanup=True, view=True)


digraph Tree {
node [shape=box, style="filled", color="black", fontname="helvetica"] ;
edge [fontname="helvetica"] ;
0 [label="supplies=high <= 0.5\ngini = 0.444\nsamples = 100.0%\nvalue = [0.667, 0.333]", fillcolor="#f2c09c"] ;
1 [label="worked=no <= 0.5\ngini = 0.5\nsamples = 66.7%\nvalue = [0.5, 0.5]", fillcolor="#ffffff"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="weather=sunny <= 0.5\ngini = 0.32\nsamples = 41.7%\nvalue = [0.8, 0.2]", fillcolor="#eca06a"] ;
1 -> 2 ;
3 [label="gini = 0.0\nsamples = 33.3%\nvalue = [1.0, 0.0]", fillcolor="#e58139"] ;
2 -> 3 ;
4 [label="gini = 0.0\nsamples = 8.3%\nvalue = [0.0, 1.0]", fillcolor="#399de5"] ;
2 -> 4 ;
5 [label="gini = 0.0\nsamples = 25.0%\nvalue = [0.0, 1.0]", fillcolor="#399de5"] ;
1 -> 5 ;
6 [label="gini = 0.0\nsamples = 33.3%\nvalue = [1.0, 0.0]", fillcolor="#e58139"] ;
0 -> 6 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;
}


ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [3]:
# ! pip install graphviz